Can we predict the ratio of a game?

Bins for ratings; -3, -2,-1, 0, 1, 2, 3, 4, 5



In [170]:
import pandas as pd # for working with data
import numpy as np # for working with data
import seaborn as sns # for making visualizations
from matplotlib import pyplot as plt # for making visualizations
import sklearn as sk
import datetime as dt
import math

In [143]:
steam_df: pd.DataFrame = pd.read_csv('games_may2024_cleaned.csv')

In [144]:
steam_df.describe()

,AppID,required_age,price,dlc_count,metacritic_score,achievements,recommendations,user_score,score_rank,positive,negative,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
count,8.364600e+04,83646.000000,83646.000000,83646.000000,83646.000000,83646.000000,8.364600e+04,83646.000000,41.000000,8.364600e+04,8.364600e+04,83646.000000,83646.000000,83646.000000,83646.000000,8.364600e+04,83646.000000,8.364600e+04,83646.000000,83646.000000
mean,1.421850e+06,0.276403,7.492446,0.580542,3.451080,20.702735,1.003478e+03,0.037467,99.121951,1.276726e+03,2.072231e+02,79.663044,3.634364,65.152572,3.744925,9.927115e+01,45.985223,1.345631e+03,5.596191,18.757430
std,7.480058e+05,2.119835,13.094604,14.179160,15.685104,167.839222,2.108173e+04,1.737074,0.748168,3.057663e+04,5.792324e+03,1018.506902,74.621737,933.278070,82.321592,5.914524e+03,40.607112,3.411672e+04,22.911354,1155.763735
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,98.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-1.000000,-1.000000e+00,-1.000000,-1.000000
25%,7.922225e+05,0.000000,0.990000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,99.000000,1.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-1.000000,-1.000000e+00,-1.000000,-1.000000
50%,1.373005e+06,0.000000,4.990000,0.000000,0.000000,4.000000,0.000000e+00,0.000000,99.000000,1.100000e+01,3.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,60.000000,1.600000e+01,-1.000000,-1.000000
75%,2.014495e+06,0.000000,9.990000,0.000000,0.000000,20.000000,0.000000e+00,0.000000,100.000000,6.400000e+01,1.900000e+01,0.000000,0.000000,0.000000,0.000000,0.000000e+00,84.000000,8.700000e+01,-1.000000,-1.000000
max,2.984570e+06,21.000000,999.980000,3165.000000,97.000000,9821.000000,4.111974e+06,100.000000,100.000000,7.024836e+06,1.029779e+06,95816.000000,6836.000000,110101.000000,9874.000000,1.362469e+06,100.000000,8.071426e+06,100.000000,321958.000000


In [145]:
# Extract release data into separate columns, as well as convert release_date into a DateTime object
steam_df['release_year'] = steam_df['release_date'].apply(lambda y: int((str(y).split('-'))[0]))
steam_df['release_month'] = steam_df['release_date'].apply(lambda m: int((str(m).split('-'))[1]))
steam_df['release_day'] = steam_df['release_date'].apply(lambda d: int((str(d).split('-'))[2]))
steam_df['release_date'] = pd.to_datetime(steam_df['release_date'], format='%Y-%m-%d')

# Get current age of game
steam_df["current_age"] = dt.datetime.now() - steam_df['release_date']

In [149]:
steam_df['total_review_count'] = steam_df['positive'] + steam_df['negative']
steam_df = steam_df[steam_df['total_review_count'] >= 10]

steam_df['positive_review_percentage'] = (steam_df['positive'] / steam_df['total_review_count']) * 100.0
steam_df = steam_df[steam_df['positive_review_percentage'].isna() == False]

sorted_positive_reviews = steam_df.sort_values(by='positive_review_percentage')
sorted_positive_reviews

,AppID,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent,release_year,release_month,release_day,current_age,total_review_count,positive_review_percentage
48945,1007970,This Side (Early Access Game),2019-05-15,0,9.99,0,This Side is a 2 players co-op roguelike survi...,This Side is a 2 players co-op roguelike survi...,Fight your natural fears and help your friend ...,NaN,...,0,10,-1,-1,2019,5,15,1968 days 17:08:23.758971,13,0.0
56151,1012210,Waves Of Death VR,2020-04-17,0,19.99,0,Waves of Death VR Kill Or Be Killed! Following...,Waves of Death VR Kill Or Be Killed! Following...,Following the tragic mistake of military scien...,NaN,...,-1,-1,-1,-1,2020,4,17,1630 days 17:08:23.758971,10,0.0
54383,1203430,Rescuers2019,2019-12-28,0,0.99,0,This is a first person action shooter game，The...,This is a first person action shooter game，The...,This is a first person action shooter game，The...,NaN,...,-1,-1,-1,-1,2019,12,28,1741 days 17:08:23.758971,14,0.0
77309,1311930,Robot Invasion,2020-07-03,0,0.99,0,Robot Invasion is a side-scroller adventure ga...,Robot Invasion is a side-scroller adventure ga...,Robot Invasion is a game set in the near futur...,NaN,...,-1,-1,-1,-1,2020,7,3,1553 days 17:08:23.758971,13,0.0
49031,1374740,Isolated Life,2020-08-26,0,9.99,0,"In this world of the future, you've fallen int...","In this world of the future, you've fallen int...","In this world of the future, you've fallen int...",NaN,...,0,10,-1,-1,2020,8,26,1499 days 17:08:23.758971,10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28742,2389700,My monstrous roommate. (étude),2023-05-09,0,0.59,0,"Today you were lying on your bed, looking up a...","Today you were lying on your bed, looking up a...",There is a monster living in your apartment.,NaN,...,97,41,-1,-1,2023,5,9,513 days 17:08:23.758971,44,100.0
83553,2443000,LIDAR Survival,2023-07-02,0,2.99,0,"A first person survival horror game, explore y...","A first person survival horror game, explore y...","LIDAR Survival is a first person, survival hor...",NaN,...,-1,-1,-1,-1,2023,7,2,459 days 17:08:23.758971,13,100.0
83507,1947240,Ashen Arrows,2023-06-12,0,17.99,0,"OUR WORLD In times of great peril, even Gods m...","OUR WORLD In times of great peril, even Gods m...",Ashen Arrows is a Nordic mythology-themed VR a...,NaN,...,-1,-1,-1,-1,2023,6,12,479 days 17:08:23.758971,10,100.0
26666,2107520,Krimson,2024-03-21,0,9.99,1,Krimson is a challenging gritty rhythm platfor...,Krimson is a challenging gritty rhythm platfor...,Krimson is a challenging gritty rhythm platfor...,NaN,...,100,49,-1,-1,2024,3,21,196 days 17:08:23.758971,57,100.0


In [184]:
# I genuinely hate how I wrote this.
def calculate_steam_rating(num_reviews: int, prp: float) -> str:
    match num_reviews:
        case num_reviews if num_reviews <= 49:
            match prp:
                case prp if 0 <= prp <= 19.0:
                    return 'negative'
                case prp if  29.0 <= prp <= 39.0:
                    return 'mostly_negative'
                case prp if 40.0 <= prp <= 69.0:
                    return 'mixed'
                case prp if 70.0 <= prp <= 79.0:
                    return 'mostly_positive'
                case prp if 80.0 <= prp <= 100.0:
                    return 'positive'
        case num_reviews if 50 >= num_reviews <= 499:
            match prp:
                case prp if 0 <= prp <= 19.0:
                    return 'very_negative'
                case prp if  29.0 <= prp <= 39.0:
                    return 'mostly_negative'
                case prp if 40.0 <= prp <= 69.0:
                    return 'mixed'
                case prp if 70.0 <= prp <= 79.0:
                    return 'mostly_positive'
                case prp if 80.0 <= prp <= 100.0:
                    return 'very_positive'
        case num_reviews if 500 <= num_reviews <= 100_000_000:
            match prp:
                case prp if 0.0 <= prp <= 19.0:
                    return 'overwhelmingly_negative'
                case prp if  29.0 <= prp <= 39.0:
                    return 'mostly_negative'
                case prp if 40.0 <= prp <= 69.0:
                    return 'mixed'
                case prp if 70.0 <= prp <= 79.0:
                    return 'mostly_positive'
                case prp if 80.0 <= prp <= 94.0:
                    return 'very_positive'
                case prp if 95.0 <= prp <= 100.0:
                    return 'overwhelmingly_positive'

calculate_steam_rating(30, 90.0)
calculate_steam_rating(300, 50.0)
calculate_steam_rating(501, 96.0)

'overwhelmingly_positive'